In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import numpy as np
import matplotlib.pyplot as plt
import os
from plot_utils import *
from utils import *
from agent_env_utils import *
from config import color_maps, background_quadrants, background_specs
import warnings

from scene_helpers import get_position_range, create_config, create_directories, finalize_quadrant


In [2]:
# Define directories
data_dir = "../benchmark/quantity/relative"
final_dir = create_directories(data_dir, empty=True)

# Initialize pair of colors
color_keys = list(color_maps.keys())

random.seed(42)
random.shuffle(color_keys)

color_pairs = list(itertools.permutations(color_keys, 2))

# Define ranges and object parameters
object_shapes = ["sphere", "cube"]
object_size_ranges = {
    "sphere": (0.02, 0.06),
    "cube": (0.02, 0.06),
}
num_objects_range = (1, 7)
# ... existing imports and functions ...

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    count = 0
    outputs = []
    for background in trange(4):
        for color1, color2 in color_pairs[:10]:
            for r in range(2):  # Increase if you want more examples per color pair and background
                
                # Generate random configurations for objects
                count += 1
                num_objects1 = np.random.randint(*num_objects_range)
                num_objects2 = np.random.randint(*num_objects_range)
                configs = []
                objects_info = {color1: [], color2: []}
                
                # Generate objects for color1
                for _ in range(num_objects1):
                    obj_type = random.choice(object_shapes)
                    #position = get_position_range(background_quadrants, background, scale=1)
                    position_range = background_specs[background]
                    size = np.random.uniform(*object_size_ranges[obj_type])
                    configs.append(create_config(obj_type, color_maps[color1], size, position_range))
                    objects_info[color1].append({"type": obj_type, "size": np.round(size, 4)})
                
                # Generate objects for color2
                for _ in range(num_objects2):
                    obj_type = random.choice(object_shapes)
                    #position = get_position_range(background_quadrants, background, scale=1)
                    position_range = background_specs[background]
                    size = np.random.uniform(*object_size_ranges[obj_type])
                    configs.append(create_config(obj_type, color_maps[color2], size, position_range))
                    objects_info[color2].append({"type": obj_type, "size": np.round(size, 4)})

                configs_collate = collate_infos(configs)

                env.register_configures(configs_collate)
                
                _ = env.reset(options={"reconfigure": True})

                env.initialize_objects(background=background)
                obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))

                save_dir_final = f"{final_dir}/scene_{count}.png"
                views = [f"front{background}", f"side{background}", f"top{background}"]
                collect_and_save(env, save_dir_final, steps=1, mode=views)
                poses_final = env.get_important_obj_poses()
                sources = [f"{save_dir_final}_{view}" for view in views]
                
                outputs.append({
                    "source": sources,
                    "color1": color1,
                    "color2": color2,
                    "num_objects1": num_objects1,
                    "num_objects2": num_objects2,
                    "background": background,
                    "round": r,
                    "objects": objects_info,
                    #"poses_final": [np.round(item, 4).tolist() for item in poses_final],
                })

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

[2024-09-17 06:34:58.603] [svulkan2] [error] GLFW error: X11: Failed to open display :0
[2024-09-17 06:34:58.603] [svulkan2] [warning] Continue without GLFW.
100%|██████████| 4/4 [02:29<00:00, 37.46s/it]


In [8]:
count

80